In [4]:
! brew install lightgbm

We (and Apple) do not provide support for this old version.
It is expected behaviour that some formulae will fail to build in this old version.
It is expected behaviour that Homebrew will be buggy and slow.
Do not create any issues about this on Homebrew's GitHub repositories.
Do not create any issues even if you think this message is unrelated.
Any opened issues will be immediately closed without response.
Do not ask for help from MacHomebrew on Twitter.
You may ask for help in Homebrew's discussions but are unlikely to receive a response.
Try to figure out the problem yourself and submit a fix as a pull request.
We will review it but may or may not accept it.

==> Downloading https://ghcr.io/v2/homebrew/core/libomp/manifests/15.0.5
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libomp/blobs/sha256:b854a6c8a9e
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
##############

In [5]:
! brew install xgboost

We (and Apple) do not provide support for this old version.
It is expected behaviour that some formulae will fail to build in this old version.
It is expected behaviour that Homebrew will be buggy and slow.
Do not create any issues about this on Homebrew's GitHub repositories.
Do not create any issues even if you think this message is unrelated.
Any opened issues will be immediately closed without response.
Do not ask for help from MacHomebrew on Twitter.
You may ask for help in Homebrew's discussions but are unlikely to receive a response.
Try to figure out the problem yourself and submit a fix as a pull request.
We will review it but may or may not accept it.

==> Downloading https://ghcr.io/v2/homebrew/core/xgboost/manifests/1.7.1
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/xgboost/blobs/sha256:a4303c48b2
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
##############

In [6]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math
import re

In [7]:
sample_submission = pd.read_csv("/Users/huoyanrong/Desktop/MMF/courses/data science/mmf-data-science-2022/sample_submission.csv")
test = pd.read_csv("/Users/huoyanrong/Desktop/MMF/courses/data science/mmf-data-science-2022/test.csv")
train = pd.read_csv("/Users/huoyanrong/Desktop/MMF/courses/data science/mmf-data-science-2022/train.csv")
#Creating a copy of the train and test datasets
c_test  = test.copy()
c_train  = train.copy()

In [8]:
c_train.head()

,id,carat,cut,color,clarity,depth,table,price,x,y,z
0,0,1.01,Very Good,E,SI2,60.0,60.0,4540,6.57,6.49,3.92
1,1,1.10,Premium,H,VS2,62.5,58.0,5729,6.59,6.54,4.10
2,2,1.50,Good,E,SI2,61.5,65.0,6300,7.21,7.17,4.42
3,3,1.53,Premium,E,SI1,61.3,59.0,12968,7.40,7.35,4.52
4,4,0.84,Fair,D,SI2,64.5,60.0,2167,5.92,5.84,3.79


In [9]:
c_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.24,Ideal,G,VVS1,62.1,56.0,3.97,4.00,2.47
1,1,1.21,Very Good,F,VS2,62.9,54.0,6.78,6.82,4.28
2,2,0.50,Fair,E,SI1,61.7,68.0,5.09,5.03,3.12
3,3,0.50,Ideal,D,SI2,62.8,56.0,5.06,5.03,3.17
4,4,1.55,Ideal,E,SI2,62.3,55.0,7.44,7.37,4.61


In [10]:
c_train['train']  = 1
c_test['train']  = 0
df = pd.concat([c_train, c_test], axis=0,sort=False)
print(df)

          id  carat        cut color clarity  depth  table    price     x  \
0          0   1.01  Very Good     E     SI2   60.0   60.0   4540.0  6.57   
1          1   1.10    Premium     H     VS2   62.5   58.0   5729.0  6.59   
2          2   1.50       Good     E     SI2   61.5   65.0   6300.0  7.21   
3          3   1.53    Premium     E     SI1   61.3   59.0  12968.0  7.40   
4          4   0.84       Fair     D     SI2   64.5   60.0   2167.0  5.92   
...      ...    ...        ...   ...     ...    ...    ...      ...   ...   
10784  10784   0.53      Ideal     F     VS2   62.3   55.0      NaN  5.20   
10785  10785   0.41    Premium     G     SI1   61.3   60.0      NaN  4.75   
10786  10786   0.51    Premium     G     VS2   62.4   58.0      NaN  5.11   
10787  10787   1.08       Good     J     SI2   63.2   59.0      NaN  6.40   
10788  10788   0.30      Ideal     E    VVS2   60.9   57.0      NaN  4.32   

          y     z  train  
0      6.49  3.92      1  
1      6.54  4.10    

In [11]:
object_columns_df = df.select_dtypes(include=['object'])
numerical_columns_df = df.select_dtypes(exclude=['object'])

In [12]:
object_columns_df.dtypes

cut        object
color      object
clarity    object
dtype: object

In [13]:
numerical_columns_df.dtypes

id         int64
carat    float64
depth    float64
table    float64
price    float64
x        float64
y        float64
z        float64
train      int64
dtype: object

In [14]:
df["cut"]=df["cut"].map({
    "Ideal":5,
    "Premium":4,
    "Very Good":3,
    "Good":2,
    "Fair":1})
print(df)

          id  carat  cut color clarity  depth  table    price     x     y  \
0          0   1.01    3     E     SI2   60.0   60.0   4540.0  6.57  6.49   
1          1   1.10    4     H     VS2   62.5   58.0   5729.0  6.59  6.54   
2          2   1.50    2     E     SI2   61.5   65.0   6300.0  7.21  7.17   
3          3   1.53    4     E     SI1   61.3   59.0  12968.0  7.40  7.35   
4          4   0.84    1     D     SI2   64.5   60.0   2167.0  5.92  5.84   
...      ...    ...  ...   ...     ...    ...    ...      ...   ...   ...   
10784  10784   0.53    5     F     VS2   62.3   55.0      NaN  5.20  5.17   
10785  10785   0.41    4     G     SI1   61.3   60.0      NaN  4.75  4.81   
10786  10786   0.51    4     G     VS2   62.4   58.0      NaN  5.11  5.12   
10787  10787   1.08    2     J     SI2   63.2   59.0      NaN  6.40  6.57   
10788  10788   0.30    5     E    VVS2   60.9   57.0      NaN  4.32  4.35   

          z  train  
0      3.92      1  
1      4.10      1  
2      4.42 

In [15]:
df['color']=df['color'].map({'G':4,'E':6,'F':5,'H':3,'D':7,'I':2,'J' : 1})
df['clarity']=df['clarity'].map({'SI1' : 3, 'VS2' : 4, 'SI2': 2,'VS1' : 5,'VVS2' : 6, 'VVS1': 7, 'IF' : 8,'I1' : 1})

In [16]:
print(df)

          id  carat  cut  color  clarity  depth  table    price     x     y  \
0          0   1.01    3      6        2   60.0   60.0   4540.0  6.57  6.49   
1          1   1.10    4      3        4   62.5   58.0   5729.0  6.59  6.54   
2          2   1.50    2      6        2   61.5   65.0   6300.0  7.21  7.17   
3          3   1.53    4      6        3   61.3   59.0  12968.0  7.40  7.35   
4          4   0.84    1      7        2   64.5   60.0   2167.0  5.92  5.84   
...      ...    ...  ...    ...      ...    ...    ...      ...   ...   ...   
10784  10784   0.53    5      5        4   62.3   55.0      NaN  5.20  5.17   
10785  10785   0.41    4      4        3   61.3   60.0      NaN  4.75  4.81   
10786  10786   0.51    4      4        4   62.4   58.0      NaN  5.11  5.12   
10787  10787   1.08    2      1        2   63.2   59.0      NaN  6.40  6.57   
10788  10788   0.30    5      6        6   60.9   57.0      NaN  4.32  4.35   

          z  train  
0      3.92      1  
1      4.

In [17]:
df = pd.get_dummies(df, columns=object_columns_df.columns)

In [18]:
df = df.drop(['id',],axis=1)
df_train = df[df['train'] == 1]
df_train = df_train.drop(['train'],axis=1)


df_test = df[df['train'] == 0]
df_test = df_test.drop(['price','train'],axis=1)

In [19]:
print(df_train)
print(df_test)

       carat  depth  table    price     x     y     z  cut_1  cut_2  cut_3  \
0       1.01   60.0   60.0   4540.0  6.57  6.49  3.92      0      0      1   
1       1.10   62.5   58.0   5729.0  6.59  6.54  4.10      0      0      0   
2       1.50   61.5   65.0   6300.0  7.21  7.17  4.42      0      1      0   
3       1.53   61.3   59.0  12968.0  7.40  7.35  4.52      0      0      0   
4       0.84   64.5   60.0   2167.0  5.92  5.84  3.79      1      0      0   
...      ...    ...    ...      ...   ...   ...   ...    ...    ...    ...   
43149   1.05   62.4   59.0   4975.0  6.48  6.51  4.05      0      0      1   
43150   0.47   61.0   55.0   1617.0  5.03  5.01  3.06      0      0      0   
43151   0.33   60.3   58.0   1014.0  4.49  4.46  2.70      0      0      1   
43152   0.90   62.8   59.0   2871.0  6.13  6.03  3.82      0      0      0   
43153   1.14   60.4   58.0   6320.0  6.82  6.79  4.11      0      0      0   

       ...  color_6  color_7  clarity_1  clarity_2  clarity_3  

In [20]:
target = df_train['price']
df_train = df_train.drop(['price'],axis=1)

In [21]:
x_train,x_test,y_train,y_test = train_test_split(df_train,target,test_size=0.3,random_state=0)

In [22]:
xgb =XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=4, min_child_weight=1.5, n_estimators=2400,
             n_jobs=1, nthread=None, objective='reg:linear',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)


lgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=12000, 
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.4, 
                                       )

In [23]:
xgb.fit(x_train, y_train)
lgbm.fit(x_train, y_train,eval_metric='rmse')

[00:49:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

In [24]:
predict1 = xgb.predict(x_test)
predict = lgbm.predict(x_test)

In [25]:
predict4 = lgbm.predict(df_test)
predict3 = xgb.predict(df_test)
predict_y = ( predict3*0.45 + predict4 * 0.55)

In [26]:
submission = pd.DataFrame({
        "id": test["id"],
        "price": predict_y
    })
submission.to_csv('submission.csv', index=False)

In [ ]:
## reference: using code from Kaggle House Prices: https://www.kaggle.com/code/fedi1996/house-prices-data-cleaning-viz-and-modeling/notebook